### Level 4 - Normalized DMSO Profiles Cell painting data


#### The goal here:

-- is to determine the median score of each compound per dose based on taking the median of the correlation values between replicates of the same compound.

- Level 4 data - are replicate level data i.e. where you have multiple profiles been perturbed by the same compound (pertubagen)

[LINCS Cell painting Level 4 Dataset](https://github.com/broadinstitute/lincs-cell-painting/tree/master/profiles/2016_04_01_a549_48hr_batch1)

In [1]:
import os
import pathlib
import pandas as pd
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from pycytominer import feature_select
from statistics import median
import random
sns.set_style("darkgrid")
from scipy import stats
import pickle

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)

In [2]:
profile_dir = 'D:\cell_painting_profiles\profiles'

In [3]:
os.listdir(profile_dir) ##files in profiles

['2016_04_01_a549_48hr_batch1',
 'cell_count',
 'media',
 'profile.py',
 'profile_utils.py',
 'profiling_pipeline.py',
 'README.md']

In [4]:
def get_normalized_dmso_files(profile_dir):
    """
    This function gets all normalized dmso Level-4 LINCS 
    Cell Painting .csv files from the profiles directory
    """
    normalized_dmso_csv_files = []
    for root, dirs, files in os.walk(profile_dir, topdown=False):
        for file in files:
            if file.endswith('normalized_dmso.csv.gz'):
                normalized_dmso_csv_files.append(os.path.join(root, file))
                
    return normalized_dmso_csv_files

In [5]:
normalized_dmso_lvl4_files = get_normalized_dmso_files(profile_dir)

In [6]:
len(normalized_dmso_lvl4_files)

136

In [7]:
df_level4 = pd.concat(map(pd.read_csv, normalized_dmso_lvl4_files)).reset_index(drop=True)

In [8]:
df_level4.shape

(52223, 1809)

In [9]:
len(df_level4['Metadata_Plate'].unique())

136

- We have 136 plates * 384 wells; in each plate we have 384 wells

In [10]:
df_level4.head()

,Metadata_plate_map_name,Metadata_broad_sample,Metadata_mg_per_ml,Metadata_mmoles_per_liter,Metadata_solvent,Metadata_pert_id,Metadata_pert_mfc_id,Metadata_pert_well,Metadata_pert_id_vendor,Metadata_cell_id,...,Nuclei_Texture_Variance_DNA_5_0,Nuclei_Texture_Variance_ER_10_0,Nuclei_Texture_Variance_ER_20_0,Nuclei_Texture_Variance_ER_5_0,Nuclei_Texture_Variance_Mito_10_0,Nuclei_Texture_Variance_Mito_20_0,Nuclei_Texture_Variance_Mito_5_0,Nuclei_Texture_Variance_RNA_10_0,Nuclei_Texture_Variance_RNA_20_0,Nuclei_Texture_Variance_RNA_5_0
0,C-7161-01-LM6-022,DMSO,0.0,0.0,DMSO,NaN,NaN,A01,NaN,A549,...,2.48340,1.38270,1.83470,0.958340,1.95550,0.92532,2.01230,-1.66110,-2.1516,-0.24654
1,C-7161-01-LM6-022,DMSO,0.0,0.0,DMSO,NaN,NaN,A02,NaN,A549,...,1.98420,1.77350,1.27920,1.269200,1.52950,1.18580,1.27550,-0.42297,1.3687,-0.36283
2,C-7161-01-LM6-022,DMSO,0.0,0.0,DMSO,NaN,NaN,A03,NaN,A549,...,0.99096,1.24930,1.35630,0.935370,1.16050,0.64569,0.62145,-0.93012,-0.6928,-0.92103
3,C-7161-01-LM6-022,DMSO,0.0,0.0,DMSO,NaN,NaN,A04,NaN,A549,...,1.31620,0.35212,0.94296,0.059474,0.29241,0.20163,0.13188,-1.52970,-1.7573,-1.56990
4,C-7161-01-LM6-022,DMSO,0.0,0.0,DMSO,NaN,NaN,A05,NaN,A549,...,0.92864,0.27655,0.43260,-0.005407,0.83840,0.75010,0.32070,0.57696,0.4506,1.73270


In [11]:
dose_liter = df_level4['Metadata_mmoles_per_liter'].unique().tolist()

In [12]:
dose_liter

[0.0,
 10.0,
 3.3333,
 1.1111,
 0.37037,
 0.12345999999999999,
 0.041152,
 20.0,
 8.106,
 2.702,
 0.9006700000000001,
 0.30022,
 0.10007,
 0.033358,
 19.999000000000002,
 1.0,
 0.33333,
 0.11110999999999999,
 0.037037,
 0.012346,
 0.0041152,
 5.7741,
 1.9247,
 0.64157,
 0.21386,
 0.071285,
 0.023762000000000002,
 4.1701,
 1.39,
 0.46334,
 0.15445,
 0.05148200000000001,
 0.017161000000000003,
 2.0,
 0.66667,
 0.22221999999999997,
 0.074074,
 0.024691,
 0.0082305,
 10.05,
 3.3499,
 1.1166,
 0.37221,
 0.12407,
 0.041356,
 10.716,
 3.5721,
 1.1907,
 0.3969,
 0.1323,
 0.0441,
 4.5176,
 1.5059,
 0.50195,
 0.16732,
 0.055772,
 0.018591,
 4.8003,
 1.6001,
 0.53337,
 0.17779,
 0.059264,
 0.019755,
 11.2,
 3.7333,
 1.2444,
 0.41481,
 0.13827,
 0.04609,
 9.5837,
 3.1946,
 1.0649,
 0.35495,
 0.11832000000000001,
 0.039439,
 11.547,
 3.8489,
 1.2830000000000001,
 0.42766000000000004,
 0.14255,
 0.047517000000000004,
 10.072000000000001,
 3.3574,
 1.1191,
 0.37305,
 0.12435,
 0.04145,
 6.7937,
 2.26

- We have 93 unique doses across the level 4 dataset, we are going to **recode the doses to 8 distinct doses**, this means we are going to assign this 93 unique doses to the nearest 8 distinct doses.

| Dose | Dose Recode |
| :--: | :---------: |
| 0 (DMSO) | 0 |
| ~0.04 | 1 |
| ~0.12 | 2 |
| ~0.37 | 3 |
| ~1.11 | 4 |
| ~3.33 | 5 |
| ~10 | 6 |
| ~20 | 7 |

In [13]:
def recode_dose(dose_value):
    """This function recode the doses in Level-4 data to 8 distinct dose classes"""
    
    doses = [0.04,0.12,0.37,1.11,3.33,10.0,20.0,25.0]
    for x in range(len(doses)-1):
        if (dose_value > 0.0) & (dose_value <= 0.04):
            dose_value = 0.04
        elif doses[x] <= round(dose_value,2) < doses[x+1]:
            dose_value = doses[x]
    return dose_value

In [14]:
df_level4['Metadata_dose_recode'] = df_level4['Metadata_mmoles_per_liter'].apply(recode_dose)

In [15]:
df_level4['Metadata_dose_recode'].unique()

array([ 0.  , 10.  ,  3.33,  1.11,  0.37,  0.12,  0.04, 20.  ])

In [16]:
def feature_selection(df_lvl4): 
    """
    Perform feature selection by dropping columns with null values 
    (greater than 384 i.e. equivalent to one plate worth of cell profiles) 
    and highly correlated values from the data.
    """
    metadata_columns = [x for x in df_lvl4.columns if (x.startswith("Metadata_"))]
    df_lvl4_metadata = df_lvl4[metadata_columns].copy()
    df_lvl4_features = df_lvl4.drop(metadata_columns, axis = 1)
    null_cols = [col for col in df_lvl4_features.columns if df_lvl4_features[col].isnull().sum() > 384]
    df_lvl4_features.drop(null_cols, axis = 1, inplace=True)
    df_lvl4_features = feature_select(df_lvl4_features, operation=["correlation_threshold", "variance_threshold"])
    
    for col in df_lvl4_features.columns:
        if df_lvl4_features[col].isnull().sum():
            df_lvl4_features[col].fillna(value=df_lvl4_features[col].mean(), inplace = True)
            
    df_meta_info = df_lvl4_metadata[['Metadata_broad_sample', 'Metadata_pert_id', 'Metadata_Plate', 'Metadata_Well',
                                     'Metadata_broad_id', 'Metadata_moa', 'Metadata_dose_recode']].copy()
    df_lvl4_new = pd.concat([df_meta_info, df_lvl4_features], axis=1)
    
    return df_lvl4_new

In [17]:
df_level4_new = feature_selection(df_level4)

In [18]:
df_level4_new.shape

(52223, 752)

In [19]:
def merge_dataframe(df, pertinfo_file):
    """
    This function merge aligned L1000 and Cell painting Metadata information dataframe 
    with the Level-4 data, change the values of the Metadata_dose_recode column 
    and create a new column 'replicate_name' that represents each replicate in the dataset
    """ 
    df_pertinfo = pd.read_csv(pertinfo_file)
    df_lvl4_new = df.merge(df_pertinfo, on='Metadata_broad_sample', how = 'outer')
    no_cpds_df = df_lvl4_new[df_lvl4_new['pert_iname'].isnull()].copy().reset_index(drop = True)
    df_lvl4_new.drop(df_lvl4_new[df_lvl4_new['pert_iname'].isnull()].index, inplace = True)
    df_lvl4_new.reset_index(drop= True, inplace = True)
    df_lvl4_new['Metadata_dose_recode'] = df_lvl4_new['Metadata_dose_recode'].map({0.0:0,0.04:1,0.12:2,0.37:3,1.11:4,
                                                                                   3.33:5,10.0:6,20.0:7})
    df_lvl4_new['replicate_name'] = ['replicate_' + str(x) for x in range(df_lvl4_new.shape[0])]
    
    return df_lvl4_new, no_cpds_df

In [20]:
pertinfo_file = '../aligned_moa_CP_L1000.csv'

In [21]:
df_level4_new, df_level4_no_cpds = merge_dataframe(df_level4_new, pertinfo_file)

In [22]:
##list of "Broad samples" WITHOUT Compounds after aligning L1000 and Cell painting MOAs
df_level4_no_cpds['Metadata_broad_sample'].unique().tolist()

['BRD-A20131130-001-01-7',
 'BRD-M98279124-300-01-1',
 'BRD-K87278688-001-01-0',
 'BRD-K21547160-001-01-4',
 'BRD-A44448661-001-04-8',
 'BRD-K41438959-001-01-7',
 'BRD-A37288617-003-02-2',
 'BRD-K52080565-001-09-2',
 'BRD-K73395020-001-02-3',
 'BRD-K01192156-001-02-7',
 'BRD-A84045418-001-03-1',
 'BRD-K60623809-001-02-0',
 'BRD-K51033547-003-02-6']

In [23]:
def get_median_score(cpds_list, df):
    """
    This function calculates the median score for each compound based on its replicates
    """
    
    cpds_median_score = {}
    for cpd in cpds_list:
        cpd_replicates = df[df['pert_iname'] == cpd].copy()
        cpd_replicates.drop(['Metadata_broad_sample', 'Metadata_pert_id', 'Metadata_dose_recode', 'Metadata_Plate',
                             'Metadata_Well', 'Metadata_broad_id', 'Metadata_moa', 'broad_id', 
                             'pert_iname', 'moa', 'replicate_name'], axis = 1, inplace = True)
        cpd_replicates_corr = cpd_replicates.astype('float64').T.corr(method = 'spearman').values
        if len(cpd_replicates_corr) == 1:
            median_val = 1
        else:
            median_val = median(list(cpd_replicates_corr[np.triu_indices(len(cpd_replicates_corr), k = 1)]))
        
        cpds_median_score[cpd] = median_val
        
    return cpds_median_score

In [24]:
def check_compounds(cpd_med_score, df):
    """
    Check if all distinct compounds in the Level-4 dataframe are present 
    in the cpd_med_score dictionary, if not add the compounds as keys to the dictionary 
    and give them a null value.
    """
    cpd_list = df['pert_iname'].unique().tolist()
    cpd_keys = cpd_med_score.keys()
    for cpd in cpd_list:
        if cpd not in cpd_keys:
            cpd_med_score[cpd] = np.nan
            
    return cpd_med_score

In [25]:
def get_cpd_medianscores(df):
    
    """This function computes median scores for all compounds found in the Level-4 dataframe PER DOSE (1-6)"""
    
    dose_list = list(set(df['Metadata_dose_recode'].unique().tolist()))[1:7]
    
    for dose in dose_list:
        df_dose = df[df['Metadata_dose_recode'] == dose].copy()
        cpds_list = df_dose['pert_iname'].unique().tolist()
        cpds_median_score = get_median_score(cpds_list, df_dose)
        cpds_median_score = check_compounds(cpds_median_score, df)
        sorted_med_score = {key:value for key, value in sorted(cpds_median_score.items(), key=lambda item: item[0])}
        if dose == 1:
            df_cpd_med_score = pd.DataFrame.from_dict(sorted_med_score, orient='index', columns = ['dose_1'])
        else:
            df_cpd_med_score['dose_' + str(dose)] = sorted_med_score.values()
            
    return df_cpd_med_score

In [26]:
df_cpd_med_score = get_cpd_medianscores(df_level4_new)

In [27]:
df_cpd_med_score.head(10)

,dose_1,dose_2,dose_3,dose_4,dose_5,dose_6
10-DEBC,0.060405,0.206655,0.055344,0.030245,0.398169,0.618065
"16,16-dimethylprostaglandin-e2",0.363895,0.432220,0.277582,NaN,NaN,NaN
17-hydroxyprogesterone-caproate,0.511133,0.329249,0.224179,0.287033,0.325095,0.883269
2-iminobiotin,0.123800,0.184245,0.105822,0.155812,0.029073,0.053605
2-methoxyestradiol,0.306166,0.289920,0.486815,0.812630,0.851861,0.909993
"3,3'-diindolylmethane",0.187089,0.309071,0.234518,NaN,NaN,NaN
3-amino-benzamide,0.421004,0.293440,0.459122,0.286109,0.488442,0.447365
3-deazaadenosine,0.077982,-0.008292,-0.014438,-0.105789,-0.018326,0.300607
ABT-737,0.265878,0.329012,0.531166,0.633499,0.719171,0.872330
AEE788,0.829277,0.291022,0.486297,0.527541,0.891382,0.901936


In [28]:
def drop_cpds_with_null(df):
    """
    This function drop compounds with median scores of 1 
    or null values in any of the dose points (1-6)
    """
    cpds_with_null = []
    for cpd in df.index:
        if any(df.loc[cpd] == 1) | any(df.loc[cpd].isnull()):
            cpds_with_null.append(cpd)
    df.drop(cpds_with_null, axis = 0, inplace = True)
    
    return df

In [29]:
df_cpd_med_score = drop_cpds_with_null(df_cpd_med_score)

In [30]:
df_cpd_med_score.head(10)

,dose_1,dose_2,dose_3,dose_4,dose_5,dose_6
10-DEBC,0.060405,0.206655,0.055344,0.030245,0.398169,0.618065
17-hydroxyprogesterone-caproate,0.511133,0.329249,0.224179,0.287033,0.325095,0.883269
2-iminobiotin,0.123800,0.184245,0.105822,0.155812,0.029073,0.053605
2-methoxyestradiol,0.306166,0.289920,0.486815,0.812630,0.851861,0.909993
3-amino-benzamide,0.421004,0.293440,0.459122,0.286109,0.488442,0.447365
3-deazaadenosine,0.077982,-0.008292,-0.014438,-0.105789,-0.018326,0.300607
ABT-737,0.265878,0.329012,0.531166,0.633499,0.719171,0.872330
AEE788,0.829277,0.291022,0.486297,0.527541,0.891382,0.901936
AICA-ribonucleotide,0.206447,0.002640,0.024904,-0.023533,0.085683,0.158885
AKT-inhibitor-1-2,0.403620,0.337821,0.468273,0.553683,0.278457,0.462571


In [31]:
def no_of_replicates_per_cpd(df, df_lvl4):
    """This function computes the numbers of replicates for each compound (cpd_size)"""
    
    dose_list = list(set(df_lvl4['Metadata_dose_recode'].unique().tolist()))[1:7]
    cpds_size = {}
    for cpd in df.index:
        num_of_replicates = 0
        for dose in dose_list:
            df_dose = df_lvl4[df_lvl4['Metadata_dose_recode'] == dose].copy()
            cpd_replicates = df_dose[df_dose['pert_iname'] == cpd].copy()
            num_of_replicates += cpd_replicates.shape[0]
        cpd_replicate_length = num_of_replicates // len(dose_list)
        cpds_size[cpd] = cpd_replicate_length
    df['cpd_size'] = cpds_size.values()
    
    return df

In [32]:
df_cpd_med_score = no_of_replicates_per_cpd(df_cpd_med_score, df_level4_new)

In [33]:
df_cpd_med_score.shape

(1538, 7)

In [34]:
def save_to_csv(df, path, file_name, compress=None):
    """saves dataframes to csv"""
    
    if not os.path.exists(path):
        os.mkdir(path)
    
    df.to_csv(os.path.join(path, file_name), index=False, compression=compress)

In [35]:
save_to_csv(df_cpd_med_score.reset_index().rename({'index':'cpd'}, axis = 1), 
            'cellpainting_lvl4_cpd_replicate_datasets', 'cpd_replicate_median_scores.csv')

In [36]:
save_to_csv(df_level4_new, 'cellpainting_lvl4_cpd_replicate_datasets', 
            'cp_level4_cpd_replicates.csv.gz', compress="gzip")